## Anchor Tabular Explainer for SKLearn Income Model

In [1]:
!cat ./models/income.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income
spec:
  storageUri: "gs://seldon-models/scv2/examples/mlserver_1.2.1/income/classifier"
  requirements:
  - sklearn


In [2]:
!seldon model load -f ./models/income.yaml

{}


In [3]:
!seldon model status income -w ModelAvailable

{}


In [4]:
!seldon model infer income \
  '{"inputs": [{"name": "predict", "shape": [1, 12], "datatype": "FP32", "data": [[47,4,1,1,1,3,4,1,0,0,40,9]]}]}' 

{
	"model_name": "income_1",
	"model_version": "1",
	"id": "f1cadfd0-24b8-4dc8-aede-cf3c1dd0973c",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1,
				1
			],
			"datatype": "INT64",
			"parameters": null,
			"data": [
				0
			]
		}
	]
}


In [5]:
!cat ./models/income-explainer.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income-explainer
spec:
  storageUri: "gs://seldon-models/scv2/examples/income/explainer"
  explainer:
    type: anchor_tabular
    modelRef: income


In [6]:
!seldon model load -f ./models/income-explainer.yaml

{}


In [7]:
!seldon model status income-explainer -w ModelAvailable

{}


In [8]:
!seldon model infer income-explainer \
  '{"inputs": [{"name": "predict", "shape": [1, 12], "datatype": "FP32", "data": [[47,4,1,1,1,3,4,1,0,0,40,9]]}]}'

{
	"model_name": "income-explainer_1",
	"model_version": "1",
	"id": "67460faa-a429-464f-8108-269ae21c024d",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "explanation",
			"shape": [
				1,
				1
			],
			"datatype": "BYTES",
			"parameters": {
				"content_type": "str",
				"headers": null
			},
			"data": [
				"{\"meta\": {\"name\": \"AnchorTabular\", \"type\": [\"blackbox\"], \"explanations\": [\"local\"], \"params\": {\"seed\": 1, \"disc_perc\": [25, 50, 75], \"threshold\": 0.95, \"delta\": 0.1, \"tau\": 0.15, \"batch_size\": 100, \"coverage_samples\": 10000, \"beam_size\": 1, \"stop_on_first\": false, \"max_anchor_size\": null, \"min_samples_start\": 100, \"n_covered_ex\": 10, \"binary_cache_size\": 10000, \"cache_margin\": 1000, \"verbose\": false, \"verbose_every\": 1, \"kwargs\": {}}, \"version\": \"0.8.0\"}, \"data\": {\"anchor\": [\"Relationship = Own-child\", \"Marital Status = Never-Married\"], \"precision\": 0.966666666666666

In [9]:
!seldon model unload income-explainer

{}


In [10]:
!seldon model unload income

{}


## Anchor Text Explainer for SKLearn Movies Sentiment Model

In [11]:
!cat ./models/moviesentiment.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: sentiment
spec:
  storageUri: "gs://seldon-models/scv2/examples/moviesentiment/classifier"
  requirements:
  - sklearn


In [14]:
!seldon model load -f ./models/moviesentiment.yaml

{}


In [15]:
!seldon model status sentiment -w ModelAvailable

{}


In [16]:
!seldon model infer sentiment \
  '{"parameters": {"content_type": "str"}, "inputs": [{"name": "foo", "data": ["I am good"], "datatype": "BYTES","shape": [1]}]}' 

{
	"model_name": "sentiment_1",
	"model_version": "1",
	"id": "6c514a89-2859-4835-97b0-a73336233726",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1,
				1
			],
			"datatype": "INT64",
			"parameters": null,
			"data": [
				1
			]
		}
	]
}


In [17]:
!cat ./models/moviesentiment-explainer.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: sentiment-explainer
spec:
  storageUri: "gs://seldon-models/scv2/examples/moviesentiment/explainer"
  explainer:
    type: anchor_text
    modelRef: sentiment


In [18]:
!seldon model load -f ./models/moviesentiment-explainer.yaml

{}


In [19]:
!seldon model status sentiment-explainer -w ModelAvailable

{}


In [20]:
!seldon model infer sentiment-explainer \
  '{"parameters": {"content_type": "str"}, "inputs": [{"name": "foo", "data": ["I am good"], "datatype": "BYTES","shape": [1]}]}' 

{
	"model_name": "sentiment-explainer_1",
	"model_version": "1",
	"id": "1d5859a9-9d1b-4edc-b1c2-b40f8369c804",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "explanation",
			"shape": [
				1,
				1
			],
			"datatype": "BYTES",
			"parameters": {
				"content_type": "str",
				"headers": null
			},
			"data": [
				"{\"meta\": {\"name\": \"AnchorText\", \"type\": [\"blackbox\"], \"explanations\": [\"local\"], \"params\": {\"seed\": 0, \"sample_proba\": 0.5}, \"version\": \"0.9.0\"}, \"data\": {\"anchor\": [\"good\"], \"precision\": 1.0, \"coverage\": 0.5006, \"raw\": {\"feature\": [2], \"mean\": [1.0], \"precision\": [1.0], \"coverage\": [0.5006], \"examples\": [{\"covered_true\": [\"I UNK good\", \"I UNK good\", \"I am good\", \"I UNK good\", \"UNK am good\", \"I UNK good\", \"UNK UNK good\", \"I UNK good\", \"I UNK good\", \"UNK am good\"], \"covered_false\": [], \"uncovered_true\": [], \"uncovered_false\": []}], \"all_precision\": 0, \

In [21]:
!seldon model unload sentiment-explainer

{}


In [22]:
!seldon model unload sentiment

{}
